In [1]:
import cv2
import numpy as np
import matplotlib.pylab as plt
import glob

In [41]:
# 동영상 파일 경로
video_path = './video1.mp4'

cap = cv2.VideoCapture(video_path)

video_file = video_path.split('/')[-1]
video_name = video_file.split('.')[0]

# 동영상 캡처 객체 생성
if cap.isOpened() :
    # 프레임 수 구하기
    fps = cap.get(cv2.CAP_PROP_FPS)
    # print(fps) # 30.0028139633606
    delay = int(1000/fps)
    print(delay) # 33
    
    # index = 1
    while True :
        ret, img = cap.read()                      # 다음 프레임 읽기
        if ret :                                   # 프레임 읽기 정상 
## ---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------- ## 
            img2 = img.copy()
            img3 = img.copy()
            h, w = img.shape[:2]

            # HSV 영상으로 변환
            hsv = cv2.cvtColor(img2, cv2.COLOR_BGR2HSV)
            img2 = cv2.bilateralFilter(img, 9, 105, 105)
            r, g, b = cv2.split(img2)
            equalize1= cv2.equalizeHist(r)
            equalize2= cv2.equalizeHist(g)
            equalize3= cv2.equalizeHist(b)
            equalize = cv2.merge((equalize1, equalize2, equalize3))
            img2 = equalize

            # 색상별 영역 지정
            red1 = np.array([0, 50, 50])
            red2 = np.array([15, 255, 255])
            red3 = np.array([165, 50, 50])
            red4 = np.array([180, 255, 255])

            # 색상에 따른 마스크 생성
            mask_red1 = cv2.inRange(hsv, red1, red2)
            mask_red2 = cv2.inRange(hsv, red3, red4)
            mask_red = mask_red1 + mask_red2

            numOfLabels, img_label, stats, centroids = cv2.connectedComponentsWithStats(mask_red)
            
            for idx, centroid in enumerate(centroids) :
                if stats[idx][0] == 0 and stats[idx][1] == 0 :
                    continue

                if np.any(np.isnan(centroid)) :
                    continue

                x, y, w, h, area = stats[idx]
                centerX, centerY = int(centroid[0]), int(centroid[1])
                # cv2.rectangle(img, (x, y), (x+w, y+h), (0, 255, 0), 1)
                
                ## 1. 크기 제한
                if area > 150 and abs(w-h) < 5 : 
                    # cv2.rectangle(img, (x, y), (x+w, y+h), (0, 0, 0), 2)
                    # cv2.circle(img, (centerX, centerY), 2, (0, 0, 255), 5)
                    
                    # 원 검출
                    detected_img = img2[y:y+h, x:x+w]
                    gray_detected_img = cv2.cvtColor(detected_img, cv2.COLOR_BGR2GRAY)
                    # gray_detected_img = cv2.resize(gray_detected_img, (gray_detected_img.shape[1]*5, gray_detected_img.shape[0]*5))
                    
                    ## 2. 파라미터 값 수정
                    circles = cv2.HoughCircles(gray_detected_img, cv2.HOUGH_GRADIENT, 1, 100, param1=200, param2=50, minRadius=10, maxRadius=0)
                    # circles = cv2.HoughCircles(gray_detected_img, cv2.HOUGH_GRADIENT, 1, 100, param1=50, param2=30, minRadius=10, maxRadius=0)

                    # print(circles)
                    if circles is not None :
                        circles = np.uint16(np.around(circles))
                        
                        for i in circles[0, :] :
                            # cv2.circle(gray_detected_img, (i[0], i[1]), i[2], (0, 255, 0), 2)
                            cv2.circle(img, (x+i[0], y+i[1]), i[2], (0, 255, 0), 2)
                            # cv2.circle(gray_detected_img, (i[0], i[1]), 2, (255, 0, 0), 5)
                            
                            ## 3. 프레임 사각형 크기 적절하게 수정
                            cv2.rectangle(img, (x-8, y-8), (x+w+8, y+h+8), (0, 0, 255), 2)
                            # cv2.imwrite('images3/{}_{}.jpg'.format(video_name, index), img3[y-3:y+h+3, x-3:x+w+3])
                            # index += 1
## ---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------- ##           
            cv2.imshow(video_file, img)        # 화면에 표시
            if cv2.waitKey(1) == 27 :
                cap.release()
                cv2.destroyAllWindows()
        else :
            break
else :
    print("can't open video.")

cap.release()
cv2.destroyAllWindows()

33


In [32]:
img = cv2.imread('./test.png')
img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
circles = cv2.HoughCircles(gray_detected_img, cv2.HOUGH_GRADIENT, 1, 100, param1=50, param2=30, minRadius=10, maxRadius=0)
print(circles)
for i in circles[0, :] :
    cv2.circle(img, (i[0], i[1]), i[2], (0, 255, 0), 2)
    cv2.circle(img, (i[0], i[1]), 2, (255, 0, 0), 5)
cv2.imshow('img', img)
cv2.waitKey()
cv2.destroyAllWindows()

[[[23.5 21.5 18. ]]]


In [13]:
# 컨투어 계층 트리

import cv2
import numpy as np

img = cv2.imread('./frame4.png')
img = cv2.resize(img, (img.shape[1]//2, img.shape[0]//2))
img2 = img.copy()
# 바이너리 이미지로 변환
# imgray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
imgray = mask_red
ret, imthres = cv2.threshold(imgray, 127, 255, cv2.THRESH_BINARY_INV)

# 가장 바깥 컨투어만 수집
# contour, hierarchy = cv2.findContours(imthres, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)

# 컨투어 개수와 계층 트리 출력
# print(len(contour), hierarchy)

# 모든 컨투어를 트리 계층으로 수집
contour2, hierarchy = cv2.findContours(imthres, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

# 컨투어 개수와 계층 트리 출력
print(len(contour2), hierarchy)

# 가장 바깥 컨투어만 그리기
# cv2.drawContours(img, contour, -1, (0, 255, 0), 3)
# 모든 컨투어 그리기
for idx, cont in enumerate(contour2) :
    # 랜덤한 컬러 추출
    color = [int(i) for i in np.random.randint(0, 255, 3)]
    # 컨투어 인덱스마다 랜덤한 색상으로 그리기
    cv2.drawContours(img2, contour2, idx, color, 3)
    # 컨투어 첫 좌표에 인덱스 숫자 표시
    # cv2.putText(img2, str(idx), tuple(cont[0][0]), cv2.FONT_HERSHEY_PLAIN, 1, (0, 0, 255))

# 화면 출력
# cv2.imshow('RETR_EXTERNAL', img)
cv2.imshow('RETR_TREE', img2)
cv2.waitKey()
cv2.destroyAllWindows()

933 [[[  1  -1  -1  -1]
  [  2   0  -1  -1]
  [  3   1  -1  -1]
  ...
  [932 929 931  24]
  [ -1  -1  -1 930]
  [ -1 930  -1  24]]]


In [ ]:
# 컨투어 계층 트리

import cv2
import numpy as np

# 영상 읽기
img = cv2.imread('../img/shapes_donut.png')
img2 = img.copy()
# 바이너리 이미지로 변환
imgray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
ret, imthres = cv2.threshold(imgray, 127, 255, cv2.THRESH_BINARY_INV)

# 가장 바깥 컨투어만 수집
contour, hierarchy = cv2.findContours(imthres, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)

# 컨투어 개수와 계층 트리 출력
print(len(contour), hierarchy)

# 모든 컨투어를 트리 계층으로 수집
contour2, hierarchy = cv2.findContours(imthres, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

# 컨투어 개수와 계층 트리 출력
print(len(contour2), hierarchy)

# 가장 바깥 컨투어만 그리기
cv2.drawContours(img, contour, -1, (0, 255, 0), 3)
# 모든 컨투어 그리기
for idx, cont in enumerate(contour2) :
    # 랜덤한 컬러 추출
    color = [int(i) for i in np.random.randint(0, 255, 3)]
    # 컨투어 인덱스마다 랜덤한 색상으로 그리기
    cv2.drawContours(img2, contour2, idx, color, 3)
    # 컨투어 첫 좌표에 인덱스 숫자 표시
    cv2.putText(img2, str(idx), tuple(cont[0][0]), cv2.FONT_HERSHEY_PLAIN, 1, (0, 0, 255))

# 화면 출력
cv2.imshow('RETR_EXTERNAL', img)
cv2.imshow('RETR_TREE', img2)
cv2.waitKey()
cv2.destroyAllWindows()

In [21]:
import cv2
import numpy as np


# 색상별 영역 지정
red1 = np.array([0, 50, 50])
red2 = np.array([15, 255, 255])
red3 = np.array([165, 50, 50])
red4 = np.array([180, 255, 255])

img = cv2.imread('./frame4.png')
img = cv2.resize(img, (img.shape[1]//2, img.shape[0]//2))
img2 = img.copy()

hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)

# 색상에 따른 마스크 생성
mask_red1 = cv2.inRange(hsv, red1, red2)
mask_red2 = cv2.inRange(hsv, red3, red4)
mask_red = mask_red1 + mask_red2

res_red1 = cv2.bitwise_and(img, img, mask = mask_red1)
res_red2 = cv2.bitwise_and(img, img, mask = mask_red2)
res_red = cv2.bitwise_or(res_red1, res_red2)

img = res_red2

# 바이너리 이미지로 변환
imgray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
ret, th = cv2.threshold(imgray, 127, 255, cv2.THRESH_BINARY_INV)

# # 이미지를 읽어서 그레이 스케일 및 스레시홀드 변환
# img = cv2.imread('./sample3_363.jpg')
# cv2.imshow('img', img)
# img2 = img.copy()
# imgray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
# ret, th = cv2.threshold(imgray, 127, 255, cv2.THRESH_BINARY_INV)

# 컨투어 찾기
contours, _ = cv2.findContours(th, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
for contour in contours :
    # 각 컨투어에 근사 컨투어로 단순화
    approx = cv2.approxPolyDP(contour, 0.01*cv2.arcLength(contour, True), True)
    print(approx)
    # 꼭짓점의 개수
    vertices = len(approx)
    print('vertices : ', vertices)
    
    # 중심점 찾기
    mmt = cv2.moments(contour)
    cx, cy = int(mmt['m10']/mmt['m00']), int(mmt['m01']/mmt['m00'])
    
    name = 'Unknown'
    if vertices == 3 :      # 꼭짓점 3개는 삼각형
        name = 'Triangle'
        color = (0, 255, 0)
    elif vertices == 4 :   # 꼭짓점 4개는 사각형
        x, y, w, h = cv2.boundingRect(contour)
        print(x, y, w, h)
        if abs(w-h) <= 3 : # 폭과 높이의 차이가 3보다 작으면 정사각형
            name = 'Square'
            color = (0, 125, 255)
        else :             # 폭과 높이의 차이가 3보다 크면 직사각형
            name = 'Rectangle'
            color = (0, 0, 255)
    elif vertices == 10 :  # 꼭짓점 개수 10개는 별
        name = 'Star'
        color = (255, 255, 0)
    elif vertices >= 15 :  # 꼭짓점 10개 이상이면 원
        name = 'Circle'
        color = (255, 0, 0)
        # 컨투어 그리기
        cv2.drawContours(img2, [contour], -1, color, -1)
        # 도형 이름 출력
        cv2.putText(img2, name, (cx-50, cy), cv2.FONT_HERSHEY_COMPLEX_SMALL, 1, (255, 0, 0), 2)

cv2.imshow('Input Shapes', img)
cv2.imshow('Recognizing Shapes', img2)
cv2.waitKey()
cv2.destroyAllWindows()

[[[  0   0]]

 [[  0 539]]

 [[959 539]]

 [[959   0]]]
vertices :  4
0 0 960 540


In [3]:
# 영상 파일 경로
video_path = './video1.mp4'
cap = cv2.VideoCapture(video_path)

while True :
    ret, img = cap.read()                            # ret : 프레임 존재 유무/ img : 프레임 읽기( cap.read() )

    if not ret :                                           # 비디오의 마지막 프레임 확인  
        print('can`t open video')
        break

    if W is None or H is None:                    # 프레임의 크기가 잡히지 않는 경우 프레임 크기를 잡는다.
        (H, W) = img.shape[:2]

    img2 = img.copy()
## ---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------- ## 

    # HSV 영상으로 변환
    hsv = cv2.cvtColor(img2, cv2.COLOR_BGR2HSV)
    img2 = cv2.bilateralFilter(img, 9, 105, 105)
    r, g, b = cv2.split(img2)
    equalize1= cv2.equalizeHist(r)
    equalize2= cv2.equalizeHist(g)
    equalize3= cv2.equalizeHist(b)
    equalize = cv2.merge((equalize1, equalize2, equalize3))
    img2 = equalize

    # 색상별 영역 지정
    red1 = np.array([0, 50, 50])
    red2 = np.array([15, 255, 255])
    red3 = np.array([165, 50, 50])
    red4 = np.array([180, 255, 255])

    # 색상에 따른 마스크 생성
    mask_red1 = cv2.inRange(hsv, red1, red2)
    mask_red2 = cv2.inRange(hsv, red3, red4)
    mask_red = mask_red1 + mask_red2

    numOfLabels, img_label, stats, centroids = cv2.connectedComponentsWithStats(mask_red2)


    for idx, centroid in enumerate(centroids) :
        if stats[idx][0] == 0 and stats[idx][1] == 0 :
            continue

        if np.any(np.isnan(centroid)) :
            continue

        x, y, W, H, area = stats[idx]                                       # x,y -> 좌표 / W,H -> 너비 , 높이
        centerX, centerY = int(centroid[0]), int(centroid[1])

        if area > 50 and abs( W - H ) < 5 : 
            # 원 검출
            detected_img = img2[ y : y + H , x : x + W ]
            gray_detected_img = cv2.cvtColor(detected_img, cv2.COLOR_BGR2GRAY)
            gray_detected_img = cv2.resize( gray_detected_img, (gray_detected_img.shape[1]*5, gray_detected_img.shape[0]*5) )
            circles = cv2.HoughCircles(gray_detected_img, cv2.HOUGH_GRADIENT, 1, 100,  param1=50, param2=30, minRadius=10, maxRadius=0 )                      
            
            if circles is not None :
                circles = np.uint16( np.around(circles) )

                for i in circles[0, : ] :
                    # 표지판 이미지 추출
                    frame = img[ y - 4 : y + H + 4 , x - 4 : x + W + 4 ]
                    
                    try : 
                        # bgr -> rgb 로 배열순서 변경
                        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
                        # 리사이즈 : 64 * 64
                        frame = cv2.resize(frame, (64, 64) )
                        # 예측
                        preds = model.predict(np.expand_dims(frame, axis=0))[0]

                        #  레이블 확인
                        label = lb.classes_[np.argmax(preds)]
                        text = "{}".format(label)

                        cv2.putText( img, text, ( x , y + 100 ), cv2.FONT_HERSHEY_SIMPLEX, 1.0, ( 0, 255, 0 ), 5)
                        cv2.rectangle(img, (x - 4, y - 4), (x + W + 4, y + H + 4), ( 255, 0, 0 ), 1)
                        #cv2.circle(frame, (i[0], i[1]), i[2], (0, 255, 0), 2)
                        #cv2.circle(frame, (i[0], i[1]), 2, (0, 0, 255), 5)
                    except :
                        continue
## ---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------- ##           
    cv2.imshow('video_file', img)        # 화면에 표시

    if cv2.waitKey(1) == 27 :
        cap.release()
        cv2.destroyAllWindows()

NameError: name 'W' is not defined

In [ ]:
# 동영상 파일 경로
video_path = './video1.mp4'

cap = cv2.VideoCapture(video_path)

video_file = video_path.split('/')[-1]
video_name = video_file.split('.')[0]

# 동영상 캡처 객체 생성
if cap.isOpened() :
    # 프레임 수 구하기
    fps = cap.get(cv2.CAP_PROP_FPS)
    # print(fps) # 30.0028139633606
    delay = int(1000/fps)
    print(delay) # 33
    
    index = 1
    while True :
        ret, img = cap.read()                      # 다음 프레임 읽기
        if ret :                                   # 프레임 읽기 정상 
## ---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------- ## 
            img2 = img.copy()
            img3 = img.copy()
            h, w = img.shape[:2]

            # HSV 영상으로 변환
            hsv = cv2.cvtColor(img2, cv2.COLOR_BGR2HSV)
            img2 = cv2.bilateralFilter(img, 9, 105, 105)
            r, g, b = cv2.split(img2)
            equalize1= cv2.equalizeHist(r)
            equalize2= cv2.equalizeHist(g)
            equalize3= cv2.equalizeHist(b)
            equalize = cv2.merge((equalize1, equalize2, equalize3))
            img2 = equalize

            # 색상별 영역 지정
            red1 = np.array([0, 50, 50])
            red2 = np.array([15, 255, 255])
            red3 = np.array([165, 50, 50])
            red4 = np.array([180, 255, 255])

            # 색상에 따른 마스크 생성
            mask_red1 = cv2.inRange(hsv, red1, red2)
            mask_red2 = cv2.inRange(hsv, red3, red4)
            mask_red = mask_red1 + mask_red2

            numOfLabels, img_label, stats, centroids = cv2.connectedComponentsWithStats(mask_red2)
            
            for idx, centroid in enumerate(centroids) :
                if stats[idx][0] == 0 and stats[idx][1] == 0 :
                    continue

                if np.any(np.isnan(centroid)) :
                    continue

                x, y, w, h, area = stats[idx]
                centerX, centerY = int(centroid[0]), int(centroid[1])

                if area > 50 and abs(w-h) < 5 : 
                    # 원 검출
                    detected_img = img2[y:y+h, x:x+w]
                    gray_detected_img = cv2.cvtColor(detected_img, cv2.COLOR_BGR2GRAY)
                    gray_detected_img = cv2.resize(gray_detected_img, (gray_detected_img.shape[1]*5, gray_detected_img.shape[0]*5))
                    circles = cv2.HoughCircles(gray_detected_img, cv2.HOUGH_GRADIENT, 1, 100, param1=50, param2=30, minRadius=10, maxRadius=0)
                    if circles is not None :
                        circles = np.uint16(np.around(circles))
                        
                        for i in circles[0, :] :
                            # cv2.circle(gray_detected_img, (i[0], i[1]), i[2], (0, 255, 0), 2)
                            # cv2.circle(gray_detected_img, (i[0], i[1]), 2, (0, 0, 255), 5)
                            cv2.rectangle(img, (x-3, y-3), (x+w+3, y+h+3), (255, 0, 0), 1)
                            cv2.imwrite('images3/{}_{}.jpg'.format(video_name, index), img3[y-3:y+h+3, x-3:x+w+3])
                            index += 1
## ---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------- ##           
            cv2.imshow(video_file, img)        # 화면에 표시
            cv2.waitKey(delay)                     # fps에 맞게 시간 지연
        else :
            break
else :
    print("can't open video.")

cap.release()
cv2.destroyAllWindows()